# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [1]:
! pip install -q schedule pytest # установка библиотек, если ещё не

In [2]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import schedule
import time
import json
import select
import sys
from datetime import datetime
import re
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
sys.path.append(os.path.abspath(".."))
from scraper import get_book_data

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [3]:
 def get_book_data(book_url: str) -> dict:
    """
    Извлекает полную информацию о книге со страницы Books to Scrape.

    Args:
        book_url (str): URL страницы книги.

    Returns:
        dict: Словарь с информацией о книге:
            - title (str): Название книги.
            - price (str): Цена (с налогом).
            - rating (float): Рейтинг по 5-балльной шкале.
            - availability (str): Наличие на складе.
            - description (str): Описание книги.
            - upc (str): Уникальный код товара.
            - product_type (str): Тип продукта.
            - price_excl_tax (str): Цена без налога.
            - price_incl_tax (str): Цена с налогом.
            - tax (str): Налог.
            - num_reviews (str): Количество отзывов.
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    response = requests.get(book_url)
    response.encoding = 'utf-8'
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
    else:
        print('ERROR REQUEST')

    # Название книги
    title = soup.find("div", class_="col-sm-6 product_main").h1.text.strip()

    # Цена
    price = soup.select_one("p.price_color").text.strip()

    # Рейтинг (в виде float)
    rating_class = soup.select_one("p.star-rating")["class"]
    rating_text = [cls for cls in rating_class if cls != "star-rating"][0]
    rating_map = {
        "One": 1.0,
        "Two": 2.0,
        "Three": 3.0,
        "Four": 4.0,
        "Five": 5.0
    }
    rating = rating_map.get(rating_text, None)

    # Наличие
    availability = soup.select_one("p.instock.availability").text.strip()

    # Описание (если есть)
    description_tag = soup.select_one("#product_description")
    if description_tag:
        description = description_tag.find_next_sibling("p").text.strip()
    else:
        description = "No description"

    # Таблица характеристик
    table = soup.select_one("table.table.table-striped")
    data = {row.th.text.strip(): row.td.text.strip()
            for row in table.select("tr")}

    return {
        "title": title,
        "price": price,
        "rating": rating,
        "availability": availability,
        "description": description,
        "upc": data.get("UPC"),
        "product_type": data.get("Product Type"),
        "price_excl_tax": data.get("Price (excl. tax)"),
        "price_incl_tax": data.get("Price (incl. tax)"),
        "tax": data.get("Tax"),
        "num_reviews": data.get("Number of reviews"),
    }

    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [4]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'title': 'A Light in the Attic',
 'price': '£51.77',
 'rating': 3.0,
 'availability': 'In stock (22 available)',
 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe plac

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [5]:
def scrape_books(is_save: bool = False) -> list:
    """
    Собирает данные о всех книгах с сайта Books to Scrape.

    Args:
        is_save (bool): Если True, сохраняет данные в файл books_data.txt.

    Returns:
        list: Список словарей с данными о книгах.
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    catalogue_root = "http://books.toscrape.com/catalogue/"
    page_url_template = "http://books.toscrape.com/catalogue/page-{}.html"
    page_num = 1
    books = []
    max_threads = 20  # многопоточный парсинг

    while True:
        url = page_url_template.format(page_num)
        response = requests.get(url)
        if response.status_code == 404:
            print(f"Проблема обработки страницы {page_num}"
                  f"(не существует или ошибка доступа)")
            break

        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, "html.parser")

        book_links = soup.select("article.product_pod h3 a")
        urls = []
        for link in book_links:
            relative_url = link.get("href").replace('../../../', '')
            if not relative_url:
                continue  # защита от отсутствующих ссылок
            full_url = urljoin(catalogue_root, relative_url)
            urls.append(full_url)

        with ThreadPoolExecutor(max_workers=max_threads) as executor:
            future_to_url = {executor.submit(get_book_data, url):
                             url for url in urls}
            for future in as_completed(future_to_url):
                url = future_to_url[future]
                try:
                    book_data = future.result()
                    books.append(book_data)
                except Exception as e:
                    print(f"Ошибка при обработке {url}: {e}")

        # print(f"[PAGE {page_num}]
        # Parsed {len(urls)} books,
        # total = {len(books)}")

        page_num += 1

    if is_save:
        with open("../artifacts/books_data.txt", "w", encoding="utf-8") as f:
            json.dump(books, f, ensure_ascii=False, indent=2)

    return books
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [6]:
# Проверка работоспособности функции
res = scrape_books(is_save=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

Проблема обработки страницы 51(не существует или ошибка доступа)
<class 'list'> 1000


## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [7]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
def job():
    """
    Запускает сбор данных о книгах и сохраняет результат в файл.
    Вызывает функцию scrape_books с флагом сохранения данных.
    Выводит сообщение в консоль до и после выполнения.
    """
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Выполняется сбор данных...")
    scrape_books(is_save=True)
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Сбор данных завершён!")


if __name__ == "__main__":
    schedule.every().day.at("22:15").do(job)  # поменять время на 19:00 для корректной работы
    print('Задача запланирована.')
    print('Чтобы прервать задачу, нажмите s/S и нажмите Enter.')
    try:
        while True:
            schedule.run_pending()
            if select.select([sys.stdin], [], [], 0.1)[0]:
                if sys.stdin.readline().strip().lower() in ['S', 's']:
                    print("Загрузка данных останавливается..")
                    break
            time.sleep(1)

    except KeyboardInterrupt:
        print('\nОстановка выполнена!')
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

Задача запланирована.
Чтобы прервать задачу, нажмите s/S и нажмите Enter.
[22:15:00] Выполняется сбор данных...
Проблема обработки страницы 51(не существует или ошибка доступа)
[22:16:07] Сбор данных завершён!

Остановка выполнена!


## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [8]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
# если в notebooks — поднимаемся
!pytest ../test/test_scraper.py -s

============================= test session starts ==============================
platform darwin -- Python 3.13.5, pytest-8.4.2, pluggy-1.5.0
rootdir: /Users/perceivery/Desktop/MIPT/SBooksScraper
plugins: hypothesis-6.142.1, anyio-4.10.0
collected 3 items                                                              

../test/test_scraper.py Проблема обработки страницы 51(не существует или ошибка доступа)
Найдено книг: 1000
Тип данных: list — OK
Список книг не пустой — OK
Тип данных: dict — OK
Все ключи присутствуют — OK
Ключи книги:
   - title
   - price
   - rating
   - availability
   - description
   - upc
   - product_type
   - price_excl_tax
   - price_incl_tax
   - tax
   - num_reviews
.Ожидаемое количество книг на сайте: 1000
Количество книг совпадает: 1000 — OK
.Заголовки книг корректны — OK
.

========================= 3 passed in 72.74s (0:01:12) =========================


## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```